# Homework 2 Part 2

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import KFold



import matplotlib.pyplot as plt
%matplotlib inline

#### Q1 (2 points)

a. Implement a function that takes in the *train.csv* dataset and returns a cleaned dataset without outliers and NAs.

In [2]:
train = pd.read_csv('train.csv')

#same as part 1 of homework2
def preprocessing_pipeline(train):
    ## Your code goes here
    train = train.drop(['Id'], axis=1)
      
    #removes NaNs from the data frame
    train = train.dropna()
    
    #loops through all columns and removes rows with outliers from the dataFrame
    #from HW1 solution
    for column in train:
        mu = np.mean(train[column])
        std = np.std(train[column])
        normalized_data = (train[column] - mu)/std
        indexes = normalized_data < 3
        train = train.loc[indexes, :]
        processed_train = train
        
    return processed_train

## THIS FUNCTION IS SLIGHLTY DIFFERENT AND GIVEES THE CORREECT VIF SCORES, 
# I had to adjsut it a little bit to the one above so that 2 part c would run, 
# it was giving me a weird error when i used the other one but i believe the other one is actually correct
'''
def preprocessing_pipeline(train):
    ## Your code goes here
    train = train.drop(['Id'], axis=1)
      
    #removes NaNs from the data frame
    train = train.dropna()
    
    #loops through all columns and removes rows with outliers from the dataFrame
    #from HW1 solution
    for column in train:
        mu = np.mean(train[column])
        std = np.std(train[column])
        normalized_data = (train[column] - mu)/std
        indexes = normalized_data < 3
        processed_train = train.loc[indexes, :]
        
    return processed_train
'''

cleaned_train = preprocessing_pipeline(train)
cleaned_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,12.0,0.37,0.76,4.2,0.066,7.0,38.0,1.00040,3.22,0.60,13.0,7
1,7.1,0.69,0.08,2.1,0.063,42.0,52.0,0.99608,3.42,0.60,10.2,6
2,9.6,0.50,0.36,2.8,0.116,26.0,55.0,0.99722,3.18,0.68,10.9,5
3,7.7,0.96,0.20,2.0,0.047,15.0,60.0,0.99550,3.36,0.44,10.9,5
5,11.9,0.38,0.49,2.7,0.098,12.0,42.0,1.00040,3.16,0.61,10.3,5


b. Implement a function that takes in the R<sup>2</sup> of a regression and return the VIF value. 

In [3]:

def calculate_vif(r_squared):
    vif = 1/(1-r_squared)
    return vif


c. Implement a function that takes in preprocessed data and returns a dataframe that contains the VIF for each variable. The dataframe should contain two columns: variable names and their VIFs.

In [4]:
def generate_vif_dataframe(processed_train):
    ## Your code goes here
    # initialize list of lists 
    newData = []
    
    vif_train = processed_train.drop(['quality'], axis = 1)
    columns = vif_train.columns
    
    columnArray = []
    
    for col in columns:   
        
        columnArray = [col]
        
        X = vif_train.drop(col, axis=1).values
        y = vif_train[col].values
        
        xConstantAdded = sm.add_constant(X)
        model = sm.OLS(y, xConstantAdded).fit()
        
        r2 = model.rsquared
        vif = round(calculate_vif(r2), 2)
        
        columnArray.append(vif)

        newData.append(columnArray)
  
        # Create the pandas DataFrame     
        vif_dataframe = pd.DataFrame(newData, columns = ['Variable', 'VIF']) 
        
        # print dataframe.    
    return vif_dataframe

d. Use the functions that you developed from Q1 and generate a VIF dataframe for all the variables in the wine dataset. Discuss the VIF values you find and the effect of multicollinearity on regression models.

In [5]:

## Your code goes here
generate_vif_dataframe(cleaned_train)


,Variable,VIF
0,fixed acidity,7.88
1,volatile acidity,1.88
2,citric acid,3.21
3,residual sugar,1.65
4,chlorides,1.18
5,free sulfur dioxide,1.98
6,total sulfur dioxide,2.20
7,density,6.25
8,pH,3.23
9,sulphates,1.25


In [6]:
## Your discussion goes here

#Looking at this model, any vif over 5 would show multicolinearity, 
#so in my case that would be fixed acidity and density.
#Multicolinearity means that there is little support for a regression plane and it 
#means that theere is a high corrilation between variables, and thus
#this can make it difficult to tell which variables are important to our model.

#### Q2 (4 points)

a. Use the VIF dataframe you generated from Q1 and choose a subset of variables that does not have high multicollinearity.

In [7]:

## Your code goes here
X = cleaned_train.drop(['fixed acidity', 'density', 'quality'], axis=1)
X.head()


,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,pH,sulphates,alcohol
0,0.37,0.76,4.2,0.066,7.0,38.0,3.22,0.60,13.0
1,0.69,0.08,2.1,0.063,42.0,52.0,3.42,0.60,10.2
2,0.50,0.36,2.8,0.116,26.0,55.0,3.18,0.68,10.9
3,0.96,0.20,2.0,0.047,15.0,60.0,3.36,0.44,10.9
5,0.38,0.49,2.7,0.098,12.0,42.0,3.16,0.61,10.3


b. Fit a multiple linear regression model and print a summary of the regression result.

In [8]:
y = cleaned_train["quality"]

results = sm.OLS(y, sm.add_constant(X)).fit()
results.summary()


/Users/emilydaskas/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                quality   R-squared:                       0.385
Model:                            OLS   Adj. R-squared:                  0.379
Method:                 Least Squares   F-statistic:                     68.45
Date:                Sun, 16 Feb 2020   Prob (F-statistic):           1.03e-97
Time:                        23:33:19   Log-Likelihood:                -945.07
No. Observations:                 995   AIC:                             1910.
Df Residuals:                     985   BIC:                             1959.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    4.2764      0.613      6.980      0.000       3.074       5.479
volatile acidity        -1.0243      0.153     -6.682      0.000      -1.325      -0.723
citric acid             -0.3771      0.157     -2.397      0.017      -0.686      -0.068
residual sugar           0.0088      0.024      0.363      0.717      -0.039       0.056
chlorides               -2.5190      0.999     -2.521      0.012      -4.479      -0.559
free sulfur dioxide      0.0040      0.003      1.318      0.188      -0.002       0.010
total sulfur dioxide    -0.0026      0.001     -2.536      0.011      -0.005      -0.001
pH                      -0.6287      0.173     -3.642      0.000      -0.967      -0.290
sulphates                1.4980      0.173      8.683      0.000       1.159       1.837
alcohol                  0.3245      0.023     14.287      0.000       0.280       0.369
==============================================================================
Omnibus:                        6.378   Durbin-Watson:                   2.073
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                8.372
Skew:                          -0.003   Prob(JB):                       0.0152
Kurtosis:                       3.449   Cond. No.                     2.77e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.77e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

c. Use 5-fold cross validation to validate the mean squared error (MSE) of a multiple linear regression model fit to the subset of data. (HINT: You will need to implement a function to calculate the MSE. See this link for more details: https://en.wikipedia.org/wiki/Mean_squared_error).

In [16]:
#worked with others on this problem

# kfold function will split the data "x" into "k" equal-sized data sets
def kfold(X,k):
    i = 0 
    while i < k:
        xtrain = np.split(X,k)
        return xtrain
        i += 1

# linreg function performs a multi-linear regression thru the x data sets & return the coefficients

def statsModel(X, y):
    xConstantAdded = sm.add_constant(X)
    results = sm.OLS(y, xConstantAdded).fit()
    return results.params

# Calculating yhat = constant + b1*x1 + b2*x2 + b3*x3 + b4*x4 + b5*x5 +b6*x6 +b7*x7 + b8*x8 + b9*x9
def yhat(parameters,X):
    yhat = parameters[0] + parameters[1]*X.iloc[:,0] + parameters[2]*X.iloc[:,1] + parameters[3]*X.iloc[:,2] + parameters[4]*X.iloc[:,3] + parameters[5]*X.iloc[:,4] + parameters[6]*X.iloc[:,5] + parameters[7]*X.iloc[:,6] + parameters[8]*X.iloc[:,7]+parameters[9]*X.iloc[:,8]
    return yhat

#Calculating MSE 
def mse(yi,yhat):
    n = len(yi)
    mse_score = (1/n) * np.sum((yi - yhat)**2)
    return mse_score
      
t1,t2,t3,t4,t5 = kfold(X, 5)
q1,q2,q3,q4,q5 = kfold(y, 5)


# Implementing 5-fold cross validation

#Round 1: 5th fold = test set
xtrain = [t1,t2,t3,t4]
ytrain = [q1,q2,q3,q4]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = statsModel(xtrain_set,ytrain_set)

yhat_train = yhat(p,t5)
mse1 = mse(q5,yhat_train)

#Round 2: 4th fold = test set
xtrain = [t1,t2,t3,t5]
ytrain = [q1,q2,q3,q5]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = statsModel(xtrain_set,ytrain_set)

yhat_train = yhat(p,t4)
mse2 = mse(q4,yhat_train)

#Round 3: 3rd fold = test set
xtrain = [t1,t2,t4,t5]
ytrain = [q1,q2,q4,q5]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = statsModel(xtrain_set,ytrain_set)

yhat_train = yhat(p,t3)
mse3 = mse(q3,yhat_train)

#Round 4: 2nd fold = test set
xtrain = [t1,t3,t4,t5]
ytrain = [q1,q3,q4,q5]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = statsModel(xtrain_set,ytrain_set)

yhat_train = yhat(p,t2)
mse4 = mse(q2,yhat_train)

#Round 5: 1st fold = test set
xtrain = [t2,t3,t4,t5]
ytrain = [q2,q3,q4,q5]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = statsModel(xtrain_set,ytrain_set)

yhat_train = yhat(p,t1)
mse5 = mse(q1,yhat_train)

average_mse = (mse1 + mse2 + mse3 + mse4 + mse5)/5


d. Make a prediction on the test set using the model you fit in 2b and submit to Kaggle.

In [15]:
## Your code goes here
test = pd.read_csv('test.csv', index_col = 0)

test = test[["volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide","pH", "sulphates","alcohol"]]

train = X
quality = y

new_para = statsModel(train,quality)
pred = yhat(new_para,test)

test['predicted'] = pred

sample_submission = pd.read_csv('sample_submission.csv')
y_pred = np.random.rand(test.shape[0])* 10
sample_submission.loc[:, 'Predicted'] = pred
sample_submission.head()
sample_submission.to_csv('Emily_Daskas_HW2.csv', header = True, index = False)

e. Compare the Kaggle score and the cross-validation result. Is the cross-validation result a good representation of the prediction accuracy on the test data?

In [11]:
## Your discussion here

#My kaggle score was similar to the cross validation result. My MSE was .402 while my kaggle score was .479, which I think are similar.
# I believe this would meean that the cross validation result was a good representation.

f. Discuss the difference between goodness of fit on the training set, on the validation set, and on the test set.

In [12]:
## Your discussion here

#The goodness of fit for these three differenet sets are similar. for the training set, it refers to how well the variance of training set is in the model created. 
#For thee validation se, it concurs that your model has beeen ntrainsed well, and then for the test set, it refers to how well it can
#be used on future data, since wee don't know "tomorrow".
